In [13]:
!pip install torch_geometric > /dev/null
print("Torch_geometric: installed")

In [11]:
!pip install geohash2

In [2]:
import requests
import pandas as pd
import os
import geohash2

In [4]:
# @title Download and store the dataset
if False:
  # URL of the dataset text file on GitHub
  text_file_url = 'https://raw.githubusercontent.com/Paco-Danes/DeepPoi/main/dataset_NYC.txt'
  # The directory to store the file in the colab session
  directory_path = '/content/dataset/'
  file_path = os.path.join(directory_path, 'example_dataset.txt')

  # Check if directory exists, if not, create it
  if not os.path.exists(directory_path):
      os.makedirs(directory_path)

  # Download the file
  r = requests.get(text_file_url)

  # Save the content to a file
  with open(file_path, 'w') as f:
      f.write(r.text)
  print("Text file downloaded and saved to:", file_path)

else:
  file_path = "/content/DeepPoi/dataset_NYC.txt"

In [3]:
!git clone https://github.com/Paco-Danes/DeepPoi.git

Cloning into 'DeepPoi'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 26 (delta 4), reused 4 (delta 1), pack-reused 14
Receiving objects: 100% (26/26), 44.47 MiB | 13.96 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [5]:
# Column names
columns = [
    'User_ID',
    'Venue_ID',
    'Venue_Category_ID',
    'Venue_Category_Name',
    'Latitude',
    'Longitude',
    'Timezone_Offset',
    'UTC_Time'
]
# Read the TSV file into a DataFrame
df = pd.read_csv(file_path, sep='\t', names = columns, encoding = "latin-1")
# Shape check of the DataFrame
df.shape

(227428, 8)

In [6]:
df.head()

,User_ID,Venue_ID,Venue_Category_ID,Venue_Category_Name,Latitude,Longitude,Timezone_Offset,UTC_Time
0,470,49bbd6c0f964a520f4531fe3,4bf58dd8d48988d127951735,Arts & Crafts Store,40.719810,-74.002581,-240,Tue Apr 03 18:00:09 +0000 2012
1,979,4a43c0aef964a520c6a61fe3,4bf58dd8d48988d1df941735,Bridge,40.606800,-74.044170,-240,Tue Apr 03 18:00:25 +0000 2012
2,69,4c5cc7b485a1e21e00d35711,4bf58dd8d48988d103941735,Home (private),40.716162,-73.883070,-240,Tue Apr 03 18:02:24 +0000 2012
3,395,4bc7086715a7ef3bef9878da,4bf58dd8d48988d104941735,Medical Center,40.745164,-73.982519,-240,Tue Apr 03 18:02:41 +0000 2012
4,87,4cf2c5321d18a143951b5cec,4bf58dd8d48988d1cb941735,Food Truck,40.740104,-73.989658,-240,Tue Apr 03 18:03:00 +0000 2012


In [7]:
# Function to retrieve geohash at different precisions
def get_geohash(row, precision):
    return geohash2.encode(row['Latitude'], row['Longitude'], precision=precision)

# List of precisions, 2 is not present because the whole NYC is in the same cell of precision 2 (every point same G@2 cell, i checked)
precisions = [3, 4, 5, 6]

# Add new columns for each precision
for precision in precisions:
    column_name = f'g{precision}'
    df[column_name] = df.apply(lambda row: get_geohash(row, precision), axis=1)

df.head()

,User_ID,Venue_ID,Venue_Category_ID,Venue_Category_Name,Latitude,Longitude,Timezone_Offset,UTC_Time,g3,g4,g5,g6
0,470,49bbd6c0f964a520f4531fe3,4bf58dd8d48988d127951735,Arts & Crafts Store,40.719810,-74.002581,-240,Tue Apr 03 18:00:09 +0000 2012,dr5,dr5r,dr5rs,dr5rsh
1,979,4a43c0aef964a520c6a61fe3,4bf58dd8d48988d1df941735,Bridge,40.606800,-74.044170,-240,Tue Apr 03 18:00:25 +0000 2012,dr5,dr5r,dr5r5,dr5r50
2,69,4c5cc7b485a1e21e00d35711,4bf58dd8d48988d103941735,Home (private),40.716162,-73.883070,-240,Tue Apr 03 18:02:24 +0000 2012,dr5,dr5r,dr5rw,dr5rws
3,395,4bc7086715a7ef3bef9878da,4bf58dd8d48988d104941735,Medical Center,40.745164,-73.982519,-240,Tue Apr 03 18:02:41 +0000 2012,dr5,dr5r,dr5ru,dr5ru3
4,87,4cf2c5321d18a143951b5cec,4bf58dd8d48988d1cb941735,Food Truck,40.740104,-73.989658,-240,Tue Apr 03 18:03:00 +0000 2012,dr5,dr5r,dr5ru,dr5ru2


In [8]:
print("How many different cells at different precisions does the dataset cover?\n", {x: len(df[x].unique()) for x in ['g3', 'g4', 'g5', 'g6']})

How many different cells at different precisions does the dataset cover?
 {'g3': 2, 'g4': 10, 'g5': 128, 'g6': 2483}


In [9]:
# Calculate the minimum and maximum values for latitude and longitude
min_latitude = df['Latitude'].min()
max_latitude = df['Latitude'].max()
min_longitude = df['Longitude'].min()
max_longitude = df['Longitude'].max()

# Min-max normalize latitude and longitude columns
df['Norm_Latitude'] = (df['Latitude'] - min_latitude) / (max_latitude - min_latitude)
df['Norm_Longitude'] = (df['Longitude'] - min_longitude) / (max_longitude - min_longitude)
df['Venue_ID_int'], _ = pd.factorize(df['Venue_ID'])

# Drop the original categorical column
df = df.drop(['Venue_Category_ID', 'Latitude', 'Longitude', 'Venue_ID'], axis=1)
print(df.shape)
df.head()

(227428, 11)


,User_ID,Venue_Category_Name,Timezone_Offset,UTC_Time,g3,g4,g5,g6,Norm_Latitude,Norm_Longitude,Venue_ID_int
0,470,Arts & Crafts Store,-240,Tue Apr 03 18:00:09 +0000 2012,dr5,dr5r,dr5rs,dr5rsh,0.386208,0.460596,0
1,979,Bridge,-240,Tue Apr 03 18:00:25 +0000 2012,dr5,dr5r,dr5r5,dr5r50,0.127885,0.390219,1
2,69,Home (private),-240,Tue Apr 03 18:02:24 +0000 2012,dr5,dr5r,dr5rw,dr5rws,0.377868,0.662835,2
3,395,Medical Center,-240,Tue Apr 03 18:02:41 +0000 2012,dr5,dr5r,dr5ru,dr5ru3,0.444161,0.494546,3
4,87,Food Truck,-240,Tue Apr 03 18:03:00 +0000 2012,dr5,dr5r,dr5ru,dr5ru2,0.432595,0.482464,4
